## Build a model in eager execution

Tensorflow guide 중, [eager execution](https://www.tensorflow.org/guide/eager)의 Build a model 부분의 코드입니다.

### Training 과정 요약

1. 모델을 정의한다.<br>
keras.Sequential을 이용하거나 subclass 형식으로 모델을 정의한다. subclass 형태로 모델을 정의할 경우, `__init__` 부분에 `tf.keras.Model`의 `__init__`을 상속받고 필요한 layer를 정의한다. `call` 에서 inputs을 받고 layer를 연결하여 class를 완성시킨다.


2. loss와 gradient function을 정의한다<br>
loss function은 예측값과 참값을 받아서 계산한다. 최종 계산된 loss값을 return 한다. 
gradient function은 `tf.GradientTape`를 이용하여 기록하는 방식을 사용한다. Tape 안에서 loss를 계산하고 모델의 변수를 loss로 미분한 결과값(미분값)을 return 한다.


3. optimizer를 정의한다.
optimizer를 정의한다. (ex. tf.train.Adamoptimizer)


4. Training epoch를 실행한다.
weight를 update하기 위해서, gradient function으로 미분값을 구한다. 미분값을 이용해서 `optimizer.apply_gradients` 를 실행하면서 weight update를 진행한다.(global step도 정의한다)

In [1]:
from __future__ import absolute_import, print_function, division

import tensorflow as tf
from tensorflow import keras

tf.enable_eager_execution()

import matplotlib.pyplot as plt
import numpy as np

print(tf.__version__)
print(tf.executing_eagerly())

C:\Users\home\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.12.0
True


먼저 keras dataset에서 mnist data를 불러온다. 불러온 dataset은 tf.data 형태로 변환시켜준다.

In [2]:
(train_image, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

불러온 데이터를 확인해보자. image는 28*28 크기의 1차원(gray scale) 이미지이다.<br>
label은 0~10 중 하나의 숫자로 표시되어 있는 unint8의 데이터라는 것을 확인할 수 있다.

In [3]:
print(train_image.shape)
print(train_labels.dtype)

(60000, 28, 28)
uint8


## Data preprocessing

`tf.data`로 변환하기 전에 image의 scale과 type을 변경시킨다.<br>
conv2d layer를 통과시키기 위해 [-1,28,28,1]로 확장한다.

In [4]:
train_image = tf.cast((train_image[...,tf.newaxis] / 255), tf.float32)
test_images = tf.cast((test_images[..., tf.newaxis] / 255), tf.float32)

train_labels = tf.cast(train_labels, tf.int64)
test_labels = tf.cast(test_labels, tf.int64)

In [53]:
test_images.shape

TensorShape([Dimension(10000), Dimension(28), Dimension(28), Dimension(1)])

`tf.data.Dataset`을 만든다.

In [54]:
tr_data = tf.data.Dataset.from_tensor_slices((train_image, train_labels))
tr_data = tr_data.shuffle(60000).batch(64)

test_data = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_data = test_data.batch(64)

In [50]:
tr_data

<BatchDataset shapes: ((?, 28, 28, 1), (?,)), types: (tf.float32, tf.int64)>

### Create Model

모델을 정의한다. Keras API를 이용해서 모델을 구성하며, 기존의 방법인 `keras.Sequential`과 subclass하는 functional API로 구성하면된다.

sample 모델은 2개의 Conv2D와 GlobalPooling, Dense로 구성되어있다.

모델에 Non-linearity(비선형성)을 추가하기 위해 사용되는 activation fucntion(활성화함수)에 대한 내용은 아래 사이트를 참고하자.<br>

* https://pozalabs.github.io/Activation_Function/ <br>
* [Wiki, sigmoid](https://en.wikipedia.org/wiki/Sigmoid_function),<br>
* [Wiki, relu](rectified linear unit)


In [8]:
#Case 1 : keras.Sequential
model = keras.Sequential([
    keras.layers.Conv2D(16, [3,3], activation=tf.nn.relu),
    keras.layers.Conv2D(16, [3,3], activation=tf.nn.relu),
    keras.layers.GlobalAvgPool2D(),
    keras.layers.Dense(10)
])

In [ ]:
#Case 2 : Subclass
class Model_cls(keras.Model):
    def __init__(self, num_of_class = 10):
        super(Model_cls, self).__init__()
        self.num_of_class = num_of_class
        self.Conv1 = keras.layers.Conv2D(16, [3,3], activation=tf.nn.relu)
        self.Conv2 = keras.layers.Conv2D(16, [3,3], activation=tf.nn.relu)
        self.Pooling = keras.layers.GlobalAvgPool2D()
        self.Dense = keras.layers.Dense(num_of_class)
        
    def call(self, inputs):
        x = self.Conv1(inputs)
        x = self.Conv2(x)
        x = self.Pooling(x)
        result = self.Dense(x)
        return result

`take`로 한 batch 만큼의 데이터를 불러와서 모델이 정상적으로 작동하는지 살펴보자.

In [9]:
#test model 
for x,y in tr_data.take(1):
    print(y[0:1])
    print(model(x[0:1]).numpy())

tf.Tensor([2], shape=(1,), dtype=int64)
[[-0.0452863  -0.04126997 -0.04053653  0.01430371 -0.00546843  0.01509097
   0.00997843  0.03687626  0.05108605 -0.00930554]]


**활성화 함수를 통과하면 값이 어떻게 변화하는지 확인해보자. 
Sigmoid는 0~1 사이의 값으로, relu는 0~x 사이의 값으로 변화하는 것을 확인할 수 있다.

In [10]:
a = tf.Variable([100.,10.,0.1, -10, -0.1])

In [11]:
print(tf.nn.sigmoid(a))
print(tf.nn.relu(a))

tf.Tensor([1.000000e+00 9.999546e-01 5.249792e-01 4.539787e-05 4.750208e-01], shape=(5,), dtype=float32)
tf.Tensor([100.   10.    0.1   0.    0. ], shape=(5,), dtype=float32)


## Define the loss and gradient function

loss function과 gradient function을 정의한다. loss는 softmax_crossentropy를 사용하고 `tf.GradientTape`로 grad를 계산하는 function을 정의한다.

label의 값이 one-hot이 아닌 0~9의 값이므로 sparse_softmax_cross_entropy 사용한다.

In [13]:
def loss(model, inputs, labels):
    y_pred = model(inputs)
    loss = tf.losses.sparse_softmax_cross_entropy(labels, y_pred)
    return loss

def grads(model, inputs, outputs):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, outputs)
    return loss_value, tape.gradient(loss_value, model.variables)

### Define the optimizer

training 과정에서 추가할 global_step도 선언해준다. global_step은 optimizer가 실행되면서 자동적으로 1씩 증가한다.

In [14]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

global_step = tf.train.get_or_create_global_step()

#for plot
train_loss_hist = []
train_acc_hist = []

In [15]:
global_step

<tf.Variable 'global_step:0' shape=() dtype=int64, numpy=0>

### Training step

epoch을 정의해주고 training step을 진행한다. training 진행 중에 mean 과 accuracy를 기록하기 위해 `tfe.metrics.Mean`, `tfe.metircs.Accuracy`로 history를 기록한다.

In [16]:
tfe = tf.contrib.eager

In [17]:
import time

In [51]:
start = time.time()
epoch_loss = tfe.metrics.Mean()
epoch_acc = tfe.metrics.Accuracy()

for x,y in tr_data:
    loss_value, grad = grads(model, x, y)
    optimizer.apply_gradients(zip(grad, model.variables), global_step)
    
    epoch_loss(loss_value)
    epoch_acc(tf.argmax(model(x), axis=1, output_type=tf.int64), y)
    
    train_loss_hist.append(epoch_loss.result())
    train_acc_hist.append(epoch_acc.result())
    
        
    if global_step.numpy() % 30 ==0:
        
        print("Epoch : {}, Loss : {:.3f}".format(global_step.numpy(), loss_value))

print("End training :{}s".format((time.time()-start)))

Epoch : 1170, Loss : 2.138
Epoch : 1200, Loss : 2.017
Epoch : 1230, Loss : 1.900


KeyboardInterrupt: 

## Test model

test_data를 이용해서 학습한 model의 accuracy를 출력해보자.

In [55]:
test_acc = tfe.metrics.Accuracy()

for x,y in test_data.take(1):
    test_acc(tf.argmax(model(x), axis=1, output_type=tf.int64), y)

print("Test accuracy : {:.2f}%".format(test_acc.result()))

Test accuracy : 0.38%


## Using model.fit

keras로 학습시키는 방법인 `model.fit`으로 training을 시켜보자.

In [56]:
#Case 1 : keras.Sequential
k_model = keras.Sequential([
    keras.layers.Conv2D(16, [3,3], activation=tf.nn.relu, input_shape=(28,28,1)),
    keras.layers.Conv2D(16, [3,3], activation=tf.nn.relu),
    keras.layers.GlobalAvgPool2D(),
    keras.layers.Dense(10)
])

In [57]:
k_model.compile(optimizer=tf.train.AdamOptimizer(), loss=tf.losses.sparse_softmax_cross_entropy, metrics=['accuracy'])

In [59]:
start_t = time.time()
model_hist = k_model.fit(train_image, train_labels, batch_size=64)
print("End Training, {}".format(time.time()-start_t))

Epoch 1/1
938/938 [==============================] - 109s 116ms/step - loss: 1.8922 - acc: 0.1611
End Training, 109.15613961219788


In [62]:
t_loss, acc = k_model.evaluate(test_images, test_labels)

print("Test Loss : {:.3f}, Acc : {:.2f}%".format(t_loss, acc))

313/313 [==============================] - 7s 23ms/step
Test Loss : 1.621, Acc : 0.24%
